# Generate index weights using dedicated service - Python

### Overview
Reference data encompasses a wide range of specification data about:
* financial instrument such as asset class, symbol, maturity, etc.
* counterparties such as issuer information.
* pricing such as Open, high low and close prices.

The challenge with reference data is that it tends to be sourced from multiple sources: internal, counterparties and providers.

Reference data dedicated service is a result of various data sources and fields sourcing, validation, cross-checking and normalization.<br>

This sample requests **on-demand** exchange data and enables fields mapping by calling a dedicated static data service, making available clean data.

### Inputs/outputs
Index weights sample is designed to help you searching and requesting index components reference data by using a wide range of commonly used fields.

It takes an index code as per input filter and returns index components and weights.

### Services used
This sample uses *gRPC requests* in order to retrieve index components and weights from the hosted service. The queried endpoint in this script are:
* *ComponentsService*: to directly retrieve index data objects from the server

### Modules required
1. Systemathics packages:
    * *systemathics.apis.services.indices.v1*
2. Open source packages
    * *googleapis-common-protos*
    * *protobuf*
    * *grpcio*
    * *pandas*
    * *plotly* and *matplotlib* as per display packages
    
***

# Run symbology queries

### Step 1: Install packages and import them

In [ ]:
pip install googleapis-common-protos protobuf grpcio pandas plotly matplotlib systemathics.apis

In [ ]:
import os
import grpc
import pandas as pd
import google.protobuf as pb
import systemathics.apis.services.indices.v1.components_pb2 as indices
import systemathics.apis.services.indices.v1.components_pb2_grpc as indices_service
import systemathics.apis.helpers.token_helpers as token_helpers
import systemathics.apis.helpers.channel_helpers as channel_helpers

### Step 2: Prepare API requests
The following code snippets retrieve authentication token to be used in upcomming API requests:

In [ ]:
token = token_helpers.get_token()
display(token)

### Step 3: Create and process request

The following code snippet enables to select the **index** by its *Name*:

In [ ]:
index = 'cac 40'

The following code snippets call the service, generate the request and return the reply: 

In [ ]:
# generate the indices request
request = indices.ComponentsRequest( 
    identifier = index
)

In [ ]:
try:
    # open a gRPC channel
    with channel_helpers.get_grpc_channel() as channel:  
        
        # instantiate the indices service
        service = indices_service.ComponentsServiceStub(channel)
        
        # process the indices request
        response = service.Components(
            request = request, 
            metadata = [('authorization', token)]
        )
except grpc.RpcError as e:
    display(e.code().name)
    display(e.details())

### Step 4: Visualize index data

#### 4.1 Retrieve components and weights

In [ ]:
# create the pandas dataframe for indices' components and weights
tickers = [c.identifier.ticker for c in response.components]
exchanges =[c.identifier.exchange for c in response.components]
weights = [c.weight for c in response.components]

d = {'Ticker': tickers, 'Exchange': exchanges, 'Weight': weights }
df = pd.DataFrame(data=d)
df = df.sort_values(by = 'Weight')

#### 4.2 Plot components and weights with matplotlib

In [ ]:
from matplotlib import pyplot as plt
import numpy as np

fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
ax.axis('equal')
ax.pie(df['Weight'], labels = df['Ticker'],autopct='%1.2f%%')
plt.legend(loc = 'upper left', title = '{0} components and weights'.format(index), bbox_to_anchor=(1.1, 1))
fig.set_size_inches(10,10)
plt.show()

#### 4.3 Plot components and weights with plotly

In [ ]:
import plotly.express as px

fig = px.pie(df, values='Weight', names='Ticker')
fig.update_traces(textposition='inside')
fig.update_layout(uniformtext_minsize=12, uniformtext_mode='hide', width=800, height=800)
fig.show()

#### 4.4 Export index components and weights
The following code snippet exports index components and weights to a *csv file*:

In [ ]:
df.to_csv('{0} weights.csv'.format(index), index=False)